In [22]:
# Imports
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os
import re
import time
import csv
import json
import random
import requests
import pandas as pd
import numpy as np

from datetime import date, timedelta, datetime

import selenium
from selenium import webdriver

#### Start the browser


In [23]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = False
browser = webdriver.Chrome(executable_path = 'C:/Users/somes/OneDrive/Documents/chromedrive/chromedriver.exe')

### login page ###

In [3]:
from selenium.webdriver.support.ui import WebDriverWait

In [24]:
#url = 'https://www.estimize.com/users/sign_in'
browser.get("https://www.estimize.com/users/sign_in")
browser.find_element_by_id("user_login").send_keys('Analyst_8910981')
browser.find_element_by_id("user_password").send_keys('nZQ9uiQdBPm6UQN')

browser.find_element_by_class_name("button-special.guest-submit").click()


## TEst if logged in

WebDriverWait(driver=browser, timeout=10).until(
    lambda x: x.execute_script("return document.readyState === 'complete'")
)
error_message = "Incorrect username or password."
# get the errors (if there are)
errors = browser.find_elements_by_class_name("flash_alert")
# print the errors optionally
# for e in errors:
#     print(e.text)
# if we find that error message within errors, then login is failed
if any(error_message in e.text for e in errors):
    print("[!] Login failed")
else:
    print("[+] Login successful")

[+] Login successful


### fetch url links for scraping sites ###

Define the tickers

In [25]:
tkrs = pd.read_csv('tickers50.csv', index_col=None, header=None)[0].to_list()
tkrs

['GPC',
 'ALV',
 'GNTX',
 'VNE',
 'TOWR',
 'DAN',
 'SMP',
 'ALSN',
 'DORM',
 'SRI',
 'BWA',
 'LKQ',
 'APTV',
 'CTB',
 'THRM',
 'VC',
 'GT',
 'LEA',
 'AXL',
 'TEN',
 'SUP',
 'SHYF',
 'CORE',
 'HZN',
 'MOD',
 'IEP',
 'MGA',
 'MPAA',
 'FOXF',
 'WGO',
 'TSLA',
 'HOG',
 'F',
 'FCAU',
 'RACE',
 'GM',
 'HMC',
 'NIO',
 'THO',
 'WGO',
 'POOL',
 'EDU',
 'TAL',
 'LRN',
 'CSV',
 'ZVO',
 'SCI',
 'WW',
 'BID',
 'RGS']

##### make things to lower

In [26]:
for i in range(len(tkrs)):
    tkrs[i] = tkrs[i].lower()
print(tkrs)

['gpc', 'alv', 'gntx', 'vne', 'towr', 'dan', 'smp', 'alsn', 'dorm', 'sri', 'bwa', 'lkq', 'aptv', 'ctb', 'thrm', 'vc', 'gt', 'lea', 'axl', 'ten', 'sup', 'shyf', 'core', 'hzn', 'mod', 'iep', 'mga', 'mpaa', 'foxf', 'wgo', 'tsla', 'hog', 'f', 'fcau', 'race', 'gm', 'hmc', 'nio', 'tho', 'wgo', 'pool', 'edu', 'tal', 'lrn', 'csv', 'zvo', 'sci', 'ww', 'bid', 'rgs']


In [27]:
len(tkrs)

50

In [7]:
 for value in tkrs:
    k = 'https://www.estimize.com/'+str(value)+'/fq3-2021?metric_name=eps&chart=table'
    print(k)   #ln = browser.get('https://www.estimize.com/value/fq1-2020?metric_name=eps&chart=table')

https://www.estimize.com/GPC/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/ALV/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/GNTX/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/COG/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/TOWR/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/DAN/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/SMP/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/ALSN/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/DORM/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/SRI/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/BWA/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/LKQ/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/APTV/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/CTB/fq3-2021?metric_name=eps&chart=table
https://www.estimize.com/THRM/fq3-2021?metric_name=eps&chart=table
http

#### Scrape and save data for means and estimize data

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Define and Start scraping by XPATH for estimates.

In [11]:
dbq1 = {}

In [15]:
from lxml import html
import requests
from multiprocessing import Process


for i in range(len(tkrs)):
    lnk ='https://www.estimize.com/'+str(tkrs[i])+'/fq1-2020?metric_name=eps&chart=table'
    browser.get(lnk)
    WebDriverWait(browser, 20)
    elements = {}
    elements['Ticker'] = tkrs[i]
    elements['Name'] = browser.find_element_by_xpath('//*[@class = "release-header-information-description"]/a').text
    elements['Sector'] = browser.find_element_by_xpath('//*[@class = "release-header-information-breadcrumb"]/span/a[1]/span').text
    elements['Industry'] = browser.find_element_by_xpath('//*[@class = "release-header-information-breadcrumb"]/span[2]/a[1]/span').text
    elements['followers'] = browser.find_element_by_xpath('//*[@class = "estimize-row summary-sub-title"][1]').text
    elements['Analysts'] = browser.find_element_by_xpath('//*[@class = "estimize-column separator-right"][2]/a').text
    elements['estimize q321'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[1]/td[12]').text
    elements['estimize q221'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[1]/td[11]').text
    elements['estimize q420'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[1]/td[10]').text
    elements['estimize q320'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[1]/td[9]').text
    elements['estimize q220'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[1]/td[8]').text
    elements['estimize q120'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[1]/td[7]').text
    #no. of estimates
    
    #mean
    elements['estimize mean q321'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[2]/td[12]').text
    elements['estimize mean q221'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[2]/td[11]').text
    elements['estimize mean q420'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[2]/td[10]').text
    elements['estimize mean q320'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[2]/td[9]').text
    elements['estimize mean q220'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[2]/td[8]').text
    elements['estimize mean q120'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-estimize"]/tr[2]/td[7]').text
    #wall street
    elements['wall st q321'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-wall-street"]/tr[1]/td[12]').text
    elements['wall st q221'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-wall-street"]/tr[1]/td[11]').text
    elements['wall st q121'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-wall-street"]/tr[1]/td[10]').text
    elements['wall st q420'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-wall-street"]/tr[1]/td[9]').text
    elements['wall st q320'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-wall-street"]/tr[1]/td[8]').text
    elements['wall st q220'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-wall-street"]/tr[1]/td[7]').text
    #actuals
    elements['actual q321'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-actual"]/tr[1]/td[12]').text
    elements['actual q221'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-actual"]/tr[1]/td[11]').text
    elements['actual q121'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-actual"]/tr[1]/td[10]').text
    elements['actual q420'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-actual"]/tr[1]/td[9]').text
    elements['actual q320'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-actual"]/tr[1]/td[8]').text
    elements['actual q220'] = browser.find_element_by_xpath('//*[@class = "rel-chart-tbl-group rel-chart-tbl-group-actual"]/tr[1]/td[7]').text
    
    dbq1[i] = elements
    data2 = pd.DataFrame.from_records(dbq1)
    

    

KeyboardInterrupt: 

In [10]:
dbq1

NameError: name 'dbq1' is not defined

## Scrape for analyst of a particular stock

In [72]:
data1={} 
datainfor = {}

In [21]:
data2 = pd.DataFrame([])
data3 = pd.DataFrame([])
datainfor = {}

In [29]:
for i in range(len(tkrs)):
    lnk ='https://www.estimize.com/'+str(tkrs[i])+'/fq1-2020?metric_name=eps&chart=table'
    browser.get(lnk)#get link
    data2 = pd.DataFrame.from_records(datainfor)
    data4 = data2.T
    data3 = data3.append(data4,ignore_index=True) #save the database
    datainfor = {} #dictionary for data in for
    #find element and scroll
    try:
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@class = "estimates-tbl-analyst"]/a[3]/p')))
        browser.execute_script("window.scrollTo(0, 500)")
    except:
        continue
    for k in range(1,100):
        #fetch and save data
        try:
            datas = {}
            ana_estimate ='//*[@id = "estimates"]//tbody[2]//tr['+str(k)+']//td[6]'
            ana_confidence='//*[@id = "estimates"]//tbody[2]//tr['+str(k)+']//td[7]'
            ana_rank = '//*[@id = "estimates"]//tbody[2]//tr['+str(k)+']//td[4]'
            ana_name= '//*[@id = "estimates"]//tbody[2]//tr['+str(k)+']//td[3]/a[2]//strong'
            ana_id = '//*[@id = "estimates"]//tbody[2]//tr['+str(k)+']//td[3]/a[3]/p'
            datas['Ticker'] = tkrs[i]
            datas['Analyst Id'] = browser.find_element_by_xpath(ana_id).text
            datas['Analyst Name'] = browser.find_element_by_xpath(ana_name).text
            datas['Analyst Rank'] = browser.find_element_by_xpath(ana_rank).text
            datas['Analyst Confidence'] = browser.find_element_by_xpath(ana_confidence).text
            datas['Analyst Confidence'] = browser.find_element_by_xpath(ana_estimate).text
            #print(datas)
            #print(k)
            datainfor[k] = datas
            #print(datainfor[k])   


        except:

            continue
    

                

1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
8
1
2
3
4
1
2
3
4
5
1
2
3
4
5
6
7
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
5
1
2
3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
1
2
3
4
5
6
7
8
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1
2
3
4
5
6
1
2
3
4
5
6
1
2
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
1
2
3
4
5
1
2
3
4
1
2
3
4
5
6
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [30]:
datainfor

{1: {'Ticker': 'rgs',
  'Analyst Id': 'BillB1210',
  'Analyst Name': 'Bill',
  'Analyst Rank': '1',
  'Analyst Confidence': '0.22'},
 2: {'Ticker': 'rgs',
  'Analyst Id': 'Jefferies_186',
  'Analyst Name': 'Stephanie Wissink',
  'Analyst Rank': '2',
  'Analyst Confidence': '0.16'},
 3: {'Ticker': 'rgs',
  'Analyst Id': 'QuantTrader007',
  'Analyst Name': 'Frederick Tremblay',
  'Analyst Rank': '3',
  'Analyst Confidence': '0.11'}}

In [31]:
data3

,Ticker,Analyst Id,Analyst Name,Analyst Rank,Analyst Confidence
0,gpc,weinhart,Rick Weinhart,1,0.89
1,gpc,QuantTrader007,Frederick Tremblay,2,1.11
2,gpc,BillB1210,Bill,3,1.12
3,gpc,benjenholdings,Ben Jen,4,1.12
4,gpc,shendricks,Scott Hendricks,5,1.21
...,...,...,...,...,...
480,ww,BillB1210,Bill,6,-0.21
481,ww,Analyst_5740786,Analyst_5740786,7,-0.21
482,ww,benjenholdings,Ben Jen,8,-0.22
483,ww,QuantTrader007,Frederick Tremblay,9,-0.22


In [80]:
with open('50 analysts_fin.json', 'w') as outfile:
    json.dump(data1, outfile, indent=4)

In [32]:
data3.to_csv('stock_analyst_data.csv')

Get analyst list

In [31]:
for i in range(len(tkrs)):
    lnk ='https://www.estimize.com/'+str(tkrs[i])+'/fq1-2020?metric_name=eps&chart=table'
    browser.get(lnk)
    

    try:
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@class = "estimates-tbl-analyst"]/a[3]/p')))
    except:
        continue
    for k in range(1,100):
        datas = {}
        try:
            datas = {}
            ana_id = '//*[@id = "estimates"]//tbody[2]//tr['+str(k)+']//td[3]/a[3]/p'
           
            datas = browser.find_element_by_xpath(ana_id).text

            print(datas)
            datainfor[k] = datas
        except:
            
            continue
        data1[i] = datainfor

weinhart
QuantTrader007
BillB1210
benjenholdings
shendricks
PM
Analyst_8075381
Analyst_5795851
QuantTrader007
BillB1210
Spekoliunas
Analyst_2777191
benjenholdings
dalton
Buck_138258
Buck_504550
New_Moon
R_C12R
AAFED
benjenholdings
Analyst_2777191
ChengSun
BillB1210
DenizMustafa
Analyst_2057501
QuantTrader007
Jefferies_174
TIMUCIN_S
BillB1210
QuantTrader007
Analyst_2777191
QuantTrader007
Sentinel
Roth_10
BillB1210
GHS_3
Analyst_6421073
Drogness
Oppenheimer_44
Analyst_2777191
BillB1210
QuantTrader007
benjenholdings
Analyst_2777191
QuantTrader007
Neil_Martin
R_C12R
maggieb
Buck_389348
QuantTrader007
AAFED
jfffrmm
Spekoliunas
Analyst_5795851
BillB1210
dalton
benjenholdings
Old_Mayor
Analyst_2777191
Analyst_8075381
PM
PCLA
rama2050
QuantTrader007
BillB1210
benjenholdings
PM
Analyst_2777191
BillB1210
QuantTrader007
benjenholdings
And_pod_13
AAFED
QuantTrader007
PM
Oppenheimer_45
Oppenheimer_44
BillB1210
Spekoliunas
maggieb
Analyst_2777191
New_Moon
Analyst_5650290
lorenzo85
benjenholdings
Ana

In [24]:
with open('idlist.json', 'w') as outfile:
    json.dump(data1, outfile, indent=4)